# 🧠 Modelo Compacto de Lenguaje con Wikipedia y DistilGPT-2
Este notebook entrena un modelo de lenguaje pequeño basado en `distilgpt2` utilizando artículos de Wikipedia.

In [ ]:
# Paso 1: Instalación de dependencias
!pip install transformers datasets gradio wikipedia

In [ ]:
# Paso 2: Importación de librerías necesarias
import wikipedia
import re
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling)
import gradio as gr

In [ ]:
# Paso 3: Recolección de datos desde Wikipedia
def recolectar_articulos(tema, num_articulos=5):
    wikipedia.set_lang("es")
    articulos = []
    titulos = wikipedia.search(tema, results=num_articulos)
    for titulo in titulos:
        try:
            contenido = wikipedia.page(titulo).content
            articulos.append(contenido)
        except:
            continue
    return articulos

In [ ]:
# Paso 4: Preprocesamiento básico del texto
def limpiar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'\[[^\]]*\]', '', texto)
    texto = re.sub(r'==.*?==', '', texto)
    return texto.strip()

In [ ]:
# Paso 5: Descargar y limpiar artículos
articulos = recolectar_articulos("inteligencia artificial", num_articulos=5)
print(f"Se descargaron {len(articulos)} artículos.")

textos_limpios = [limpiar_texto(t) for t in articulos]
textos_entrenamiento = textos_limpios
print(f"Se limpiaron {len(textos_entrenamiento)} textos para el entrenamiento.")

In [ ]:
# Paso 6: Cargar el modelo preentrenado y corregir token de padding
modelo_id = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
modelo = AutoModelForCausalLM.from_pretrained(modelo_id)

# Solución al error de padding
tokenizer.pad_token = tokenizer.eos_token
modelo.config.pad_token_id = modelo.config.eos_token_id

In [ ]:
# Paso 7: Entrenamiento del modelo (fine-tuning)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

dataset = Dataset.from_dict({"text": textos_entrenamiento})

argumentos_entrenamiento = TrainingArguments(
    output_dir="./modelo_compacto",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
)

entrenador = Trainer(
    model=modelo,
    args=argumentos_entrenamiento,
    data_collator=data_collator,
    train_dataset=dataset
)

entrenador.train()

In [ ]:
# Paso 8: Crear interfaz con Gradio para probar el modelo
def responder(prompt):
    entradas = tokenizer(prompt, return_tensors="pt")
    salida = modelo.generate(**entradas, max_new_tokens=50)
    return tokenizer.decode(salida[0], skip_special_tokens=True)

iface = gr.Interface(fn=responder, inputs="text", outputs="text", title="Modelo Compacto de Lenguaje")
iface.launch()

✅ **Listo:** ahora podrás probar el modelo escribiendo una frase en la interfaz de Gradio que se abrirá al final.